In [1]:
from src.indexing import build_index_from_url, build_chunks_from_tree, EmbeddingManager
import numpy as np


In [2]:
tree, folder_path = await build_index_from_url("https://en.wikipedia.org/wiki/Brazil", 1)
chunks = build_chunks_from_tree(tree)
em = EmbeddingManager(index_dir = folder_path)
index = em.generate_index_from_chunks(chunks, "website_embbed_index3")
text = em.search_index(["what is the area of brazil?"], 5)

Loading model from local directory or downloading it...
Generating embeddings...
Created FAISS index with 622 vectors
Index saved to data\websites\en.wikipedia.org_wiki_Brazil\website_embbed_index3.index
Generating embeddings...


In [4]:
text[1]

array([['Brazil is the fifth largest country in the world, and third largest in the Americas, with a total area of 8,515,767.049 km2 (3,287,956 sq mi),[176] including 55,455 km2 (21,411 sq mi) of water. North to South, Brazil is also the longest country in the world, spanning 4,395 km (2,731 mi) from north to south,[13] and the only country in the world that has the equator and the Tropic',
        "Bounded by the Atlantic Ocean on the east, Brazil has a coastline of 7,491 kilometers (4,655 mi).[13] Covering roughly half of South America's land area, it borders all other countries and territories on the continent except Ecuador and Chile.[14] Brazil encompasses a wide range of tropical and subtropical landscapes, as well as wetlands, savannas, plateaus, and low mountains. It",
        "Brazil,[b] officially the Federative Republic of Brazil,[c] is the largest and easternmost country in South America. It is the world's fifth-largest country by area and the seventh largest by population,

In [ ]:
def build_starting_prompt(chunks, query):
    base_prompt = '''Using the information below, answer any question the user might have about this topic. If the answer cannot be found, write
    "I'm sorry, but i couldn't find the answer."
    '''
    for information_chunks in chunks:
        base_prompt += f"\nInformation: {information_chunks}"
    return base_prompt + f"\nUser question: {query}\nAnswer clearly and concisely."

In [ ]:
def build_conversation_prompt(context_chunks, history, query):

In [ ]:
async def first_ask_model(url, query, user):
    tree, folder_path = await build_index_from_url(url, 1)
    chunks = build_chunks_from_tree(tree)
    em = EmbeddingManager(index_dir = folder_path)
    index = em.generate_index_from_chunks(chunks, "website_embbed_index3") 
    related_chunks = em.search_index([query], 5)
    prompt = build_starting_prompt(related_chunks, query)